In [1]:
from imblearn.datasets import fetch_datasets
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
wine_quality_import = fetch_datasets()['wine_quality']

In [2]:
wine_quality = pd.DataFrame(wine_quality_import.data)

wine_quality['target'] = wine_quality_import.target

In [3]:
wine_quality.loc[wine_quality['target'] == -1, 'target'] = 0

In [4]:
X_train, X_test, y_train, y_test = train_test_split(wine_quality.drop('target',axis=1), wine_quality['target'], test_size=0.33, random_state=42)

In [25]:
# Random Forest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector

num_pipe = SimpleImputer(strategy="mean", add_indicator=True)
cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
)

preprocessor_tree = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

rf_clf = make_pipeline(
    preprocessor_tree, RandomForestClassifier()
)

In [12]:
## RANDOM PARAMETER GRID SEARCH
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(start = 250, stop = 350, num = 10)]
# Number of features to consider at every split
max_features = ['log2','sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(30, 50, num = 5)]
max_depth.append(None)

criterion = ['gini', 'entropy']
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split':min_samples_split,
               'min_samples_leaf':min_samples_leaf,
               'bootstrap': bootstrap,
                'criterion':criterion}

print(random_grid)

{'n_estimators': [250, 261, 272, 283, 294, 305, 316, 327, 338, 350], 'max_features': ['log2', 'sqrt'], 'max_depth': [30, 35, 40, 45, 50, None], 'min_samples_split': [2], 'min_samples_leaf': [1], 'bootstrap': [False], 'criterion': ['gini', 'entropy']}


In [13]:
rf_random = GridSearchCV(estimator = RandomForestClassifier(), 
                               param_grid = random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [14]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 240 candidates, totalling 720 fits


KeyboardInterrupt: 

In [256]:
rf_random.best_score_

0.8956388086414308

In [257]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 45,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 327}

In [501]:
brf_random = GridSearchCV(estimator = BalancedRandomForestClassifier(), 
                               param_grid = random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [ ]:
brf_random.fit(X_train, y_train)

Fitting 3 folds for each of 1440 candidates, totalling 4320 fits


exception calling callback for <Future at 0x1b798c61548 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\joblib\parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "c:\users

In [ ]:
brf_random.best_score_

In [ ]:
brf_random.best_params_

In [18]:
#Random Forest with oversampling using SMOTE and undersampling using RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline

over_n = [float(x) for x in np.arange(0.2, 0.35, 0.05)]
# RandomUnderSampler undersampling
under_n = [float(x) for x in np.arange(0.35, 0.5, 0.05)]

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
               'randomforestclassifier__min_samples_split': min_samples_split,
               'randomforestclassifier__min_samples_leaf': min_samples_leaf,
               'randomforestclassifier__bootstrap': bootstrap,
               'randomforestclassifier__criterion':criterion,
              'smote__sampling_strategy':over_n,
              'randomundersampler__sampling_strategy':under_n}

print(random_grid)

{'randomforestclassifier__n_estimators': [250, 261, 272, 283, 294, 305, 316, 327, 338, 350], 'randomforestclassifier__max_features': ['log2', 'sqrt'], 'randomforestclassifier__max_depth': [30, 35, 40, 45, 50, None], 'randomforestclassifier__min_samples_split': [2], 'randomforestclassifier__min_samples_leaf': [1], 'randomforestclassifier__bootstrap': [False], 'randomforestclassifier__criterion': ['gini', 'entropy'], 'smote__sampling_strategy': [0.2, 0.25, 0.3], 'randomundersampler__sampling_strategy': [0.35, 0.39999999999999997, 0.44999999999999996, 0.49999999999999994]}


In [19]:
# define pipeline
# oversample positive (minority) to be x% the number of negative (majority)
over = SMOTE(sampling_strategy = 0.1)
# randomly undersample negative (majority) to reduce the number of negative to x% of the positive (minority)
under = RandomUnderSampler(sampling_strategy = 0.25) 

pipeline = make_pipeline(over, under, RandomForestClassifier(n_estimators = 260,
 min_samples_split =2,
 min_samples_leaf=1,
 max_features= 'auto',
 max_depth= 90,
 bootstrap= False))

rf_sampling_random = GridSearchCV(pipeline,
                               param_grid = random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [20]:
rf_sampling_random.fit(X_train,y_train)


Fitting 3 folds for each of 2880 candidates, totalling 8640 fits


KeyboardInterrupt: 

In [ ]:
rf_sampling_random.best_score_


In [ ]:
rf_sampling_random.best_params_

In [41]:
# BALANCED BAGGING

# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(190, 230, 10)]

# Method of selecting samples for training each tree
bootstrap = [True]

over_n = [float(x) for x in np.arange(0.25, 0.35, 0.05)]
# RandomUnderSampler undersampling
under_n = [float(x) for x in np.arange(0.35, 0.45, 0.05)]

# Create the random grid
bb_random_grid = {'balancedbaggingclassifier__n_estimators': n_estimators,
                 'balancedbaggingclassifier__bootstrap': bootstrap,
                 'smote__sampling_strategy':over_n,
              'randomundersampler__sampling_strategy':under_n}


# define pipeline
# oversample positive (minority) to be x% the number of negative (majority)
over = SMOTE(sampling_strategy = 0.1)
# randomly undersample negative (majority) to reduce the number of negative to x% of the positive (minority)
under = RandomUnderSampler(sampling_strategy = 0.25) 

pipeline = make_pipeline(over, under, BalancedBaggingClassifier(base_estimator=HistGradientBoostingClassifier(random_state=42)))

In [42]:
bb_random = GridSearchCV(estimator = pipeline, 
                               param_grid = bb_random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [43]:
bb_random.fit(X_train, y_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits


KeyboardInterrupt: 

In [ ]:
bb_random.best_score_

In [234]:
bb_random.best_params_

{'n_estimators': 207, 'bootstrap': True}

In [46]:
# LOGISTIC REGRESSION
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate



num_pipe = make_pipeline(
    StandardScaler(), SimpleImputer(strategy="mean", add_indicator=True)
)
cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_linear = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

lr_clf = make_pipeline(preprocessor_linear, LogisticRegression(max_iter=1000))
lr_clf.set_params(logisticregression__class_weight="balanced")

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=2,
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler()),
                                                                  ('simpleimputer',
                                                                   SimpleImputer(add_indicator=True))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001311D2F8FC8>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant'))

In [230]:
cv_result = cross_validate(lr_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [231]:
cv_result['test_score'].mean()

0.7959038021320113

In [47]:
# Support Vector Classification with RandomOverSampling
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline 

num_pipe = make_pipeline(
    MinMaxScaler(feature_range=(0, 1))
)
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_svc = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

smote = SMOTE(sampling_strategy=0.1)

rus = RandomUnderSampler(sampling_strategy=0.2)

svc_clf = make_pipeline(preprocessor_svc, smote, rus, SVC(kernel='rbf',C=1))

In [48]:
cv_result = cross_validate(svc_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [49]:
cv_result['test_score'].mean()

0.8478555658111919

In [51]:
parameters = {'smote__sampling_strategy':[float(x) for x in np.arange(0.2, 0.35, 0.05)],
              'randomundersampler__sampling_strategy':[float(x) for x in np.arange(0.35, 0.45, 0.05)],
              'svc__kernel':('linear', 'rbf', 'poly'), 
              'svc__C':[1,10,100,1000],
              'svc__gamma':[1,0.1,0.001,0.0001], 
              'svc__degree':[1,2]}

svc_grid = GridSearchCV(svc_clf, parameters, scoring="roc_auc", cv=3)

In [52]:
svc_grid.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
svc_grid.best_score_

In [445]:
svc_grid.best_params_

{'rus__sampling_strategy': 0.2,
 'smote__sampling_strategy': 0.1,
 'svc__C': 1,
 'svc__class_weight': None,
 'svc__degree': 2,
 'svc__kernel': 'rbf'}